In [33]:
import os
import pickle
import numpy as np
import pandas as pd
import keras
from keras.datasets import mnist
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
### Downloading the dataset
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
#### Reshaping the data into 32*32*3 nupy array
x_train = x_train.reshape(50000, 3072)
x_test = x_test.reshape(10000, 3072)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')


In [9]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)


### First let's try running the model without normalizing the data and see what we get

In [10]:
batch_size = 128
num_classes = 10
epochs = 30


In [11]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(3072,)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               393344    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 394,634
Trainable params: 394,634
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 3s 61us/step - loss: 14.5035 - acc: 0.1001 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 2/30
50000/50000 [==============================] - 3s 55us/step - loss: 14.5060 - acc: 0.1000 - val_loss: 14.5063 - val_acc: 0.1000
Epoch 3/30
50000/50000 [==============================] - 3s 56us/step -

### As you can see without normalizing the data the accuracy remained constant at 0.1 for all the epochs 
### Let's see if it changes once the data is normalized

In [12]:
x_train/=255
x_test/=255

In [16]:
epochs = [20,30,40]
batch_size = [128,256,512]
dropout_rate=[0.1,0.2,0.5,1]
neurons=[128,256,512]


### Building a single layer neural network and doing grid search to get the best parameters 

In [36]:
result1=pd.DataFrame(columns=['epochs','batch_size','dropout_rate','neurons','Test_sccuracy'])

In [37]:
bestscore1=0
best_epoch1=0
best_batch_size1=0
best_dropout_rate1=0
best_neuron1=0
rownum1=0
for i in epochs:
    for j in batch_size:
        for k in dropout_rate:
            for l in neurons:
                rownum1+=1
                model = Sequential()
                model.add(Dense(10, input_shape=(3072,)))
                model.add(Activation('softmax'))
                result_1=pd.DataFrame(columns=['epochs','batch_size','dropout_rate','neurons','Test_sccuracy'])
                model.compile(loss='categorical_crossentropy',
                              optimizer=RMSprop(),
                              metrics=['accuracy'])

                history = model.fit(x_train, y_train,
                                    batch_size=j,
                                    epochs=i,
                                    verbose=1,
                                    validation_data=(x_test, y_test))
                score = model.evaluate(x_test, y_test, verbose=0)
                print('Test loss:', score[0])
                print('Test accuracy:', score[1])
                if score[1]>bestscore:
                    bestscore1=score[1]
                    best_epoch1=i
                    best_batch_size1=j
                    best_dropout_rate1=k
                    best_neuron1=l
                result1.loc[rownum1]=[i,j,k,l,score[1]]
                print("Parameters are epochs: ",i,"  batch_size: ",j,"  dropout_rate: ",k,"   neurons:", l)
                print("")
                print("<------------------------------------------------------------------------------------->")
                print("")

print("")



Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 4s 85us/step - loss: 2.1841 - acc: 0.2549 - val_loss: 2.0624 - val_acc: 0.2836
Epoch 2/20
50000/50000 [==============================] - 2s 33us/step - loss: 2.0540 - acc: 0.3038 - val_loss: 2.0733 - val_acc: 0.2846
Epoch 3/20
50000/50000 [==============================] - 1s 30us/step - loss: 2.0225 - acc: 0.3153 - val_loss: 2.2350 - val_acc: 0.2755
Epoch 4/20
50000/50000 [==============================] - 2s 30us/step - loss: 1.9991 - acc: 0.3243 - val_loss: 1.9656 - val_acc: 0.3412
Epoch 5/20
50000/50000 [==============================] - 2s 30us/step - loss: 1.9845 - acc: 0.3320 - val_loss: 1.9556 - val_acc: 0.3392
Epoch 6/20
50000/50000 [==============================] - 1s 30us/step - loss: 1.9781 - acc: 0.3334 - val_loss: 1.7939 - val_acc: 0.3703
Epoch 7/20
50000/50000 [==============================] - 1s 30us/step - loss: 1.9627 - acc: 0.3365 - val_loss: 2.3373 - val_acc

Epoch 10/20
50000/50000 [==============================] - 2s 30us/step - loss: 1.9478 - acc: 0.3462 - val_loss: 1.8867 - val_acc: 0.3507
Epoch 11/20
50000/50000 [==============================] - 2s 30us/step - loss: 1.9427 - acc: 0.3453 - val_loss: 2.2875 - val_acc: 0.2832
Epoch 12/20
50000/50000 [==============================] - 2s 30us/step - loss: 1.9415 - acc: 0.3485 - val_loss: 2.0814 - val_acc: 0.3011
Epoch 13/20
50000/50000 [==============================] - 2s 30us/step - loss: 1.9335 - acc: 0.3504 - val_loss: 2.1358 - val_acc: 0.2984
Epoch 14/20
50000/50000 [==============================] - 2s 30us/step - loss: 1.9266 - acc: 0.3508 - val_loss: 1.9902 - val_acc: 0.3488
Epoch 15/20
50000/50000 [==============================] - 2s 31us/step - loss: 1.9266 - acc: 0.3514 - val_loss: 1.9082 - val_acc: 0.3655
Epoch 16/20
50000/50000 [==============================] - 2s 30us/step - loss: 1.9218 - acc: 0.3560 - val_loss: 1.9388 - val_acc: 0.3374
Epoch 17/20
50000/50000 [=========

Epoch 19/20
50000/50000 [==============================] - 1s 27us/step - loss: 1.9194 - acc: 0.3556 - val_loss: 1.8714 - val_acc: 0.3622
Epoch 20/20
50000/50000 [==============================] - 1s 27us/step - loss: 1.9120 - acc: 0.3591 - val_loss: 1.9494 - val_acc: 0.3347
Test loss: 1.94943257713
Test accuracy: 0.3347
Parameters are epochs:  20   batch_size:  128   dropout_rate:  1    neurons: 256

<------------------------------------------------------------------------------------->

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 4s 78us/step - loss: 2.1882 - acc: 0.2566 - val_loss: 2.2020 - val_acc: 0.2570
Epoch 2/20
50000/50000 [==============================] - 1s 27us/step - loss: 2.0482 - acc: 0.3056 - val_loss: 2.2782 - val_acc: 0.2766
Epoch 3/20
50000/50000 [==============================] - 1s 27us/step - loss: 2.0182 - acc: 0.3152 - val_loss: 1.9830 - val_acc: 0.3294
Epoch 4/20
50000/50000 [=====================

Epoch 6/20
50000/50000 [==============================] - 1s 18us/step - loss: 1.9898 - acc: 0.3302 - val_loss: 2.2907 - val_acc: 0.2476
Epoch 7/20
50000/50000 [==============================] - 1s 18us/step - loss: 1.9770 - acc: 0.3313 - val_loss: 2.1089 - val_acc: 0.2979
Epoch 8/20
50000/50000 [==============================] - 1s 18us/step - loss: 1.9689 - acc: 0.3332 - val_loss: 1.9786 - val_acc: 0.3120
Epoch 9/20
50000/50000 [==============================] - 1s 18us/step - loss: 1.9676 - acc: 0.3358 - val_loss: 2.0553 - val_acc: 0.3103
Epoch 10/20
50000/50000 [==============================] - 1s 18us/step - loss: 1.9567 - acc: 0.3398 - val_loss: 2.2288 - val_acc: 0.2941
Epoch 11/20
50000/50000 [==============================] - 1s 18us/step - loss: 1.9507 - acc: 0.3410 - val_loss: 2.1006 - val_acc: 0.2909
Epoch 12/20
50000/50000 [==============================] - 1s 18us/step - loss: 1.9451 - acc: 0.3437 - val_loss: 1.9719 - val_acc: 0.3136
Epoch 13/20
50000/50000 [=============

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 3s 70us/step - loss: 2.2148 - acc: 0.2482 - val_loss: 2.4238 - val_acc: 0.2555
Epoch 2/20
50000/50000 [==============================] - 1s 18us/step - loss: 2.0683 - acc: 0.2958 - val_loss: 2.0847 - val_acc: 0.2659
Epoch 3/20
50000/50000 [==============================] - 1s 18us/step - loss: 2.0359 - acc: 0.3118 - val_loss: 2.2317 - val_acc: 0.2914
Epoch 4/20
50000/50000 [==============================] - 1s 18us/step - loss: 2.0082 - acc: 0.3175 - val_loss: 2.1123 - val_acc: 0.3075
Epoch 5/20
50000/50000 [==============================] - 1s 18us/step - loss: 1.9980 - acc: 0.3236 - val_loss: 2.1910 - val_acc: 0.2822
Epoch 6/20
50000/50000 [==============================] - 1s 18us/step - loss: 1.9900 - acc: 0.3268 - val_loss: 2.0484 - val_acc: 0.2925
Epoch 7/20
50000/50000 [==============================] - 1s 18us/step - loss: 1.9709 - acc: 0.3365 - val_loss: 1.9862 - val_acc

...

Epoch 14/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9552 - acc: 0.3388 - val_loss: 1.9551 - val_acc: 0.3245
Epoch 15/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9566 - acc: 0.3389 - val_loss: 2.0693 - val_acc: 0.2799
Epoch 16/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9571 - acc: 0.3396 - val_loss: 1.8525 - val_acc: 0.3515
Epoch 17/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9555 - acc: 0.3392 - val_loss: 1.8995 - val_acc: 0.3452
Epoch 18/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9449 - acc: 0.3406 - val_loss: 2.0150 - val_acc: 0.3471
Epoch 19/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9374 - acc: 0.3457 - val_loss: 1.7989 - val_acc: 0.3729
Epoch 20/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9398 - acc: 0.3452 - val_loss: 2.0057 - val_acc: 0.3166
Epoch 21/40
50000/50000 [=========

50000/50000 [==============================] - 1s 20us/step - loss: 2.0132 - acc: 0.3164 - val_loss: 1.9332 - val_acc: 0.3177
Epoch 7/40
50000/50000 [==============================] - 1s 20us/step - loss: 2.0050 - acc: 0.3213 - val_loss: 1.9549 - val_acc: 0.3258
Epoch 8/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9882 - acc: 0.3250 - val_loss: 2.0098 - val_acc: 0.3072
Epoch 9/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9781 - acc: 0.3283 - val_loss: 1.9804 - val_acc: 0.3234
Epoch 10/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9739 - acc: 0.3327 - val_loss: 2.1054 - val_acc: 0.2920
Epoch 11/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9735 - acc: 0.3339 - val_loss: 2.2854 - val_acc: 0.2816
Epoch 12/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9627 - acc: 0.3369 - val_loss: 2.2013 - val_acc: 0.2797
Epoch 13/40
50000/50000 [========================

50000/50000 [==============================] - 1s 22us/step - loss: 1.9070 - acc: 0.3584 - val_loss: 1.9370 - val_acc: 0.3497
Test loss: 1.93696332874
Test accuracy: 0.3497
Parameters are epochs:  40   batch_size:  512   dropout_rate:  0.2    neurons: 512

<------------------------------------------------------------------------------------->

Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 5s 103us/step - loss: 2.3161 - acc: 0.2179 - val_loss: 2.1387 - val_acc: 0.2513
Epoch 2/40
50000/50000 [==============================] - 1s 20us/step - loss: 2.1136 - acc: 0.2801 - val_loss: 1.9448 - val_acc: 0.3067
Epoch 3/40
50000/50000 [==============================] - 1s 20us/step - loss: 2.0688 - acc: 0.2921 - val_loss: 2.0363 - val_acc: 0.3026
Epoch 4/40
50000/50000 [==============================] - 1s 20us/step - loss: 2.0418 - acc: 0.3068 - val_loss: 1.9656 - val_acc: 0.3269
Epoch 5/40
50000/50000 [==============================]

50000/50000 [==============================] - 1s 20us/step - loss: 1.9220 - acc: 0.3502 - val_loss: 2.0109 - val_acc: 0.3268
Epoch 33/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9177 - acc: 0.3542 - val_loss: 1.8958 - val_acc: 0.3406
Epoch 34/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9102 - acc: 0.3578 - val_loss: 1.9082 - val_acc: 0.3470
Epoch 35/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9143 - acc: 0.3544 - val_loss: 2.0840 - val_acc: 0.3173
Epoch 36/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9047 - acc: 0.3607 - val_loss: 2.0727 - val_acc: 0.3351
Epoch 37/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9086 - acc: 0.3552 - val_loss: 2.0110 - val_acc: 0.3225
Epoch 38/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9147 - acc: 0.3575 - val_loss: 1.9342 - val_acc: 0.3582
Epoch 39/40
50000/50000 [=====================

50000/50000 [==============================] - 1s 20us/step - loss: 1.9294 - acc: 0.3516 - val_loss: 1.9146 - val_acc: 0.3294
Epoch 25/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9258 - acc: 0.3475 - val_loss: 2.0295 - val_acc: 0.3305
Epoch 26/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9313 - acc: 0.3490 - val_loss: 1.9638 - val_acc: 0.3121
Epoch 27/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9252 - acc: 0.3524 - val_loss: 1.8579 - val_acc: 0.3582
Epoch 28/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9150 - acc: 0.3523 - val_loss: 1.8997 - val_acc: 0.3373
Epoch 29/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9137 - acc: 0.3569 - val_loss: 1.9338 - val_acc: 0.3603
Epoch 30/40
50000/50000 [==============================] - 1s 20us/step - loss: 1.9221 - acc: 0.3530 - val_loss: 2.1460 - val_acc: 0.3225
Epoch 31/40
50000/50000 [=====================

In [38]:
result1

,epochs,batch_size,dropout_rate,neurons,Test_sccuracy
1,20.0,128.0,0.1,128.0,0.3256
2,20.0,128.0,0.1,256.0,0.3554
3,20.0,128.0,0.1,512.0,0.3438
4,20.0,128.0,0.2,128.0,0.3090
5,20.0,128.0,0.2,256.0,0.3294
6,20.0,128.0,0.2,512.0,0.3547
7,20.0,128.0,0.5,128.0,0.3284
8,20.0,128.0,0.5,256.0,0.3306
9,20.0,128.0,0.5,512.0,0.3321
10,20.0,128.0,1.0,128.0,0.3139


### We can see when we ran the single layer model we got the best accuracy as 36.34% 
### Most of the time the accuracy was in between 33% to 35%
### As we increased the number of neurons the accuracy increased
### increasing the number of epochs had a positive effect on the accuracy but not a major effect
### Batch Size also had really minimal change on the test accuracy
### Dropout rate when increased from 0.1 to 0.5 decreased the accuracy but as we went closet to the dropout raate 1 it started giving us the best results

### Adding one more layer and Doing hyperparameter tuning in a grid search

In [20]:
### Let's store the results to a data frame along with the hyperparameters

result=pd.DataFrame(columns=['epochs','batch_size','dropout_rate','neurons','Test_sccuracy'])

In [21]:
bestscore=0
best_epoch=0
best_batch_size=0
best_dropout_rate=0
best_neuron=0
rownum=0
for i in epochs:
    for j in batch_size:
        for k in dropout_rate:
            for l in neurons:
                rownum+=1
                model = Sequential()
                model.add(Dense(l, activation='relu', input_shape=(3072,)))
                model.add(Dropout(k))
                model.add(Dense(num_classes, activation='softmax'))

                model.summary()

                model.compile(loss='categorical_crossentropy',
                              optimizer=RMSprop(),
                              metrics=['accuracy'])

                history = model.fit(x_train, y_train,
                                    batch_size=j,
                                    epochs=i,
                                    verbose=1,
                                    validation_data=(x_test, y_test))
                score = model.evaluate(x_test, y_test, verbose=0)
                print('Test loss:', score[0])
                print('Test accuracy:', score[1])
                if score[1]>bestscore:
                    bestscore=score[1]
                    best_epoch=i
                    best_batch_size=j
                    best_dropout_rate=k
                    best_neuron=l
                result.loc[rownum]=[i,j,k,l,score[1]]
                print("Parameters are epochs: ",i,"  batch_size: ",j,"  dropout_rate: ",k,"   neurons:", l)
                print("")
                print("<------------------------------------------------------------------------------------->")
                print("")

print("")


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 128)               393344    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                1290      
Total params: 394,634
Trainable params: 394,634
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 3s 69us/step - loss: 2.1412 - acc: 0.2433 - val_loss: 1.8965 - val_acc: 0.3095
Epoch 2/20
50000/50000 [==============================] - 3s 61us/step - loss: 1.8699 - acc: 0.3242 - val_loss: 1.7640 - val_acc: 0.3690
Epoch 3/20
50000/50000 [==============================] - 3s 61us/step - los

50000/50000 [==============================] - 11s 218us/step - loss: 2.7786 - acc: 0.2500 - val_loss: 1.8657 - val_acc: 0.3270
Epoch 2/20
50000/50000 [==============================] - 11s 213us/step - loss: 1.8298 - acc: 0.3433 - val_loss: 1.8996 - val_acc: 0.3234
Epoch 3/20
50000/50000 [==============================] - 11s 212us/step - loss: 1.7475 - acc: 0.3750 - val_loss: 1.6664 - val_acc: 0.4060
Epoch 4/20
50000/50000 [==============================] - 11s 212us/step - loss: 1.7012 - acc: 0.3964 - val_loss: 1.6490 - val_acc: 0.4134
Epoch 5/20
50000/50000 [==============================] - 9s 187us/step - loss: 1.6671 - acc: 0.4109 - val_loss: 1.5944 - val_acc: 0.4371
Epoch 6/20
50000/50000 [==============================] - 10s 195us/step - loss: 1.6436 - acc: 0.4174 - val_loss: 1.6377 - val_acc: 0.4228
Epoch 7/20
50000/50000 [==============================] - 9s 190us/step - loss: 1.6229 - acc: 0.4281 - val_loss: 1.6789 - val_acc: 0.4023
Epoch 8/20
50000/50000 [================

Epoch 6/20
50000/50000 [==============================] - 6s 123us/step - loss: 1.7195 - acc: 0.3851 - val_loss: 1.6292 - val_acc: 0.4218
Epoch 7/20
50000/50000 [==============================] - 6s 128us/step - loss: 1.7023 - acc: 0.3919 - val_loss: 1.6076 - val_acc: 0.4345
Epoch 8/20
50000/50000 [==============================] - 6s 117us/step - loss: 1.6859 - acc: 0.3971 - val_loss: 1.6275 - val_acc: 0.4081
Epoch 9/20
50000/50000 [==============================] - 5s 109us/step - loss: 1.6768 - acc: 0.4026 - val_loss: 1.6022 - val_acc: 0.4316
Epoch 10/20
50000/50000 [==============================] - 6s 111us/step - loss: 1.6607 - acc: 0.4103 - val_loss: 1.6049 - val_acc: 0.4333
Epoch 11/20
50000/50000 [==============================] - 6s 128us/step - loss: 1.6502 - acc: 0.4129 - val_loss: 1.5633 - val_acc: 0.4443
Epoch 12/20
50000/50000 [==============================] - 6s 123us/step - loss: 1.6380 - acc: 0.4160 - val_loss: 1.5867 - val_acc: 0.4415
Epoch 13/20
50000/50000 [======

Epoch 15/20
50000/50000 [==============================] - 11s 221us/step - loss: 1.7651 - acc: 0.3656 - val_loss: 1.6176 - val_acc: 0.4324
Epoch 16/20
50000/50000 [==============================] - 10s 210us/step - loss: 1.7577 - acc: 0.3679 - val_loss: 1.5945 - val_acc: 0.4387
Epoch 17/20
50000/50000 [==============================] - 10s 209us/step - loss: 1.7527 - acc: 0.3725 - val_loss: 1.6942 - val_acc: 0.4047
Epoch 18/20
50000/50000 [==============================] - 10s 209us/step - loss: 1.7482 - acc: 0.3727 - val_loss: 1.6193 - val_acc: 0.4317
Epoch 19/20
50000/50000 [==============================] - 11s 221us/step - loss: 1.7430 - acc: 0.3709 - val_loss: 1.6134 - val_acc: 0.4265
Epoch 20/20
50000/50000 [==============================] - 12s 239us/step - loss: 1.7390 - acc: 0.3768 - val_loss: 1.6109 - val_acc: 0.4241
Test loss: 1.61090701103
Test accuracy: 0.4241
Parameters are epochs:  20   batch_size:  128   dropout_rate:  0.5    neurons: 512

<----------------------------

Epoch 20/20
50000/50000 [==============================] - 5s 102us/step - loss: 1.4352 - acc: 0.4944 - val_loss: 1.6752 - val_acc: 0.4309
Test loss: 1.6752453373
Test accuracy: 0.4309
Parameters are epochs:  20   batch_size:  128   dropout_rate:  1    neurons: 256

<------------------------------------------------------------------------------------->

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 512)               1573376   
_________________________________________________________________
dropout_17 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 10)                5130      
Total params: 1,578,506
Trainable params: 1,578,506
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 1000

...

Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 9s 178us/step - loss: 5.1530 - acc: 0.1734 - val_loss: 2.0297 - val_acc: 0.2453
Epoch 2/40
50000/50000 [==============================] - 6s 128us/step - loss: 2.0424 - acc: 0.2629 - val_loss: 1.8697 - val_acc: 0.3298
Epoch 3/40
50000/50000 [==============================] - 6s 128us/step - loss: 1.9376 - acc: 0.3039 - val_loss: 1.8291 - val_acc: 0.3271
Epoch 4/40
50000/50000 [==============================] - 6s 128us/step - loss: 1.8742 - acc: 0.3283 - val_loss: 1.7450 - val_acc: 0.3793
Epoch 5/40
50000/50000 [==============================] - 6s 129us/step - loss: 1.8320 - acc: 0.3470 - val_loss: 1.7498 - val_acc: 0.3704
Epoch 6/40
50000/50000 [==============================] - 7s 131us/step - loss: 1.7961 - acc: 0.3587 - val_loss: 1.7223 - val_acc: 0.3896
Epoch 7/40
50000/50000 [==============================] - 7s 130us/step - loss: 1.7747 - acc: 0.3699 - val_loss: 1.6769 - 

50000/50000 [==============================] - 4s 71us/step - loss: 1.7891 - acc: 0.3593 - val_loss: 1.6288 - val_acc: 0.4258
Epoch 24/40
50000/50000 [==============================] - 4s 71us/step - loss: 1.7856 - acc: 0.3620 - val_loss: 1.6695 - val_acc: 0.4257
Epoch 25/40
50000/50000 [==============================] - 4s 71us/step - loss: 1.7784 - acc: 0.3603 - val_loss: 1.6537 - val_acc: 0.4286
Epoch 26/40
50000/50000 [==============================] - 4s 71us/step - loss: 1.7748 - acc: 0.3616 - val_loss: 1.7465 - val_acc: 0.3742
Epoch 27/40
50000/50000 [==============================] - 4s 72us/step - loss: 1.7717 - acc: 0.3661 - val_loss: 1.6514 - val_acc: 0.4138
Epoch 28/40
50000/50000 [==============================] - 4s 71us/step - loss: 1.7672 - acc: 0.3628 - val_loss: 1.6332 - val_acc: 0.4145
Epoch 29/40
50000/50000 [==============================] - 4s 72us/step - loss: 1.7656 - acc: 0.3675 - val_loss: 1.6142 - val_acc: 0.4303
Epoch 30/40
50000/50000 [=====================

Total params: 789,258
Trainable params: 789,258
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 6s 122us/step - loss: 3.9016 - acc: 0.1758 - val_loss: 2.3135 - val_acc: 0.1982
Epoch 2/40
50000/50000 [==============================] - 3s 70us/step - loss: 2.1359 - acc: 0.2462 - val_loss: 2.2017 - val_acc: 0.2516
Epoch 3/40
50000/50000 [==============================] - 4s 70us/step - loss: 1.9649 - acc: 0.2974 - val_loss: 1.9430 - val_acc: 0.3238
Epoch 4/40
50000/50000 [==============================] - 4s 70us/step - loss: 1.8841 - acc: 0.3260 - val_loss: 1.7929 - val_acc: 0.3516
Epoch 5/40
50000/50000 [==============================] - 3s 70us/step - loss: 1.8295 - acc: 0.3484 - val_loss: 1.7399 - val_acc: 0.3839
Epoch 6/40
50000/50000 [==============================] - 4s 71us/step - loss: 1.7868 - acc: 0.3647 - val_loss: 1.8184 - val_a

Epoch 10/40
50000/50000 [==============================] - 6s 124us/step - loss: 1.6530 - acc: 0.4147 - val_loss: 1.7045 - val_acc: 0.3907
Epoch 11/40
50000/50000 [==============================] - 6s 124us/step - loss: 1.6293 - acc: 0.4250 - val_loss: 1.7266 - val_acc: 0.3867
Epoch 12/40
50000/50000 [==============================] - 6s 124us/step - loss: 1.6120 - acc: 0.4313 - val_loss: 1.6291 - val_acc: 0.4237
Epoch 13/40
50000/50000 [==============================] - 6s 124us/step - loss: 1.5912 - acc: 0.4386 - val_loss: 1.6718 - val_acc: 0.4136
Epoch 14/40
50000/50000 [==============================] - 6s 125us/step - loss: 1.5801 - acc: 0.4421 - val_loss: 1.5776 - val_acc: 0.4376
Epoch 15/40
50000/50000 [==============================] - 6s 124us/step - loss: 1.5585 - acc: 0.4516 - val_loss: 1.5854 - val_acc: 0.4349
Epoch 16/40
50000/50000 [==============================] - 6s 124us/step - loss: 1.5472 - acc: 0.4537 - val_loss: 1.5426 - val_acc: 0.4539
Epoch 17/40
50000/50000 [==

NameError: name 'best_neurons' is not defined

In [23]:
### Best epoch 256
### Dropout rate has negative effect

print(" The model with the best result has a score: ",bestscore,"  Epochs: ",best_epoch,"  Batch_size: ",best_batch_size,"  dropout_rate: ",best_dropout_rate, "  neurons: ",best_neuron )


 The model with the best result has a score:  0.4881   Epochs:  30   Batch_size:  128   dropout_rate:  1   neurons:  512


In [24]:
result

,epochs,batch_size,dropout_rate,neurons,Test_sccuracy
1,20.0,128.0,0.1,128.0,0.4299
2,20.0,128.0,0.1,256.0,0.4631
3,20.0,128.0,0.1,512.0,0.4597
4,20.0,128.0,0.2,128.0,0.4151
5,20.0,128.0,0.2,256.0,0.4554
6,20.0,128.0,0.2,512.0,0.4464
7,20.0,128.0,0.5,128.0,0.3495
8,20.0,128.0,0.5,256.0,0.4047
9,20.0,128.0,0.5,512.0,0.4241
10,20.0,128.0,1.0,128.0,0.4551


In [29]:
result[result['Test_sccuracy']>0.48]

,epochs,batch_size,dropout_rate,neurons,Test_sccuracy
12,20.0,128.0,1.0,512.0,0.4806
48,30.0,128.0,1.0,512.0,0.4881
72,30.0,512.0,1.0,512.0,0.4830
84,40.0,128.0,1.0,512.0,0.4835
102,40.0,512.0,0.2,512.0,0.4836
108,40.0,512.0,1.0,512.0,0.4837


#### We can see when we ran the single layer model we got the best accuracy as 48.81% which is a 12% change from the previous single layer model
#### Most of the time the accuracy was in between 40% to 48%
#### As we increased the number of neurons the accuracy increased
#### increasing the number of epochs had a positive effect on the accuracy but not a major effect
#### Batch Size also had really minimal change on the test accuracy
#### Dropout rate when increased from 0.1 to 0.5 decreased the accuracy but as we went closet to the dropout raate 1 it started giving us the best results

### One thing to note is that when we add a new layer it increases the time taken to build the model to almost double but since the model increased the accuracy by 12% it is acceptable 


### Adding one more layer and checking if it imporves the result
### But this time instead of doing exhaustive search for hyperparameters let's do it on the best params

In [41]:
result3=pd.DataFrame(columns=['epochs','batch_size','dropout_rate','neurons','Test_sccuracy'])

#### From our previous results we found that some hyperparameters value were giving us the best results like epochs-[30,40] , neurons[256,512], dropoutrate=[0.1,1] so let's try this model with these parameters only  

In [42]:
epochs3 = [30,40]
batch_size3 = [128,256,512]
dropout_rate3=[0.1,1]
neurons3=[256,512]


In [43]:
bestscore3=0
best_epoch3=0
best_batch_size3=0
best_dropout_rate3=0
best_neuron3=0
rownum3=0
for i in epochs3:
    for j in batch_size3:
        for k in dropout_rate3:
            for l in neurons3:
                rownum3+=1
                model3 = Sequential()
                model3.add(Dense(l, activation='relu', input_shape=(3072,)))
                model3.add(Dropout(k))
                model3.add(Dense(l, activation='relu'))
                model3.add(Dropout(k))
                model3.add(Dense(num_classes, activation='softmax'))

                model3.summary()

                model3.compile(loss='categorical_crossentropy',
                              optimizer=RMSprop(),
                              metrics=['accuracy'])

                history = model3.fit(x_train, y_train,
                                    batch_size=j,
                                    epochs=i,
                                    verbose=1,
                                    validation_data=(x_test, y_test))
                score = model3.evaluate(x_test, y_test, verbose=0)
                print('Test loss:', score[0])
                print('Test accuracy:', score[1])
                if score[1]>bestscore:
                    bestscore3=score[1]
                    best_epoch3=i
                    best_batch_size3=j
                    best_dropout_rate3=k
                    best_neuron3=l
                result3.loc[rownum3]=[i,j,k,l,score[1]]
                print("Parameters are epochs: ",i,"  batch_size: ",j,"  dropout_rate: ",k,"   neurons:", l)
                print("")
                print("<------------------------------------------------------------------------------------->")
                print("")

print("")


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_339 (Dense)            (None, 256)               786688    
_________________________________________________________________
dropout_114 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_340 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_115 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_341 (Dense)            (None, 10)                2570      
Total params: 855,050
Trainable params: 855,050
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 11s 226us/step - loss: 2.080

Epoch 11/30
50000/50000 [==============================] - 13s 262us/step - loss: 1.5098 - acc: 0.4644 - val_loss: 1.5012 - val_acc: 0.4637
Epoch 12/30
50000/50000 [==============================] - 13s 262us/step - loss: 1.4927 - acc: 0.4653 - val_loss: 1.4854 - val_acc: 0.4788
Epoch 13/30
50000/50000 [==============================] - 13s 262us/step - loss: 1.4792 - acc: 0.4733 - val_loss: 1.4942 - val_acc: 0.4719
Epoch 14/30
50000/50000 [==============================] - 13s 263us/step - loss: 1.4700 - acc: 0.4777 - val_loss: 1.4448 - val_acc: 0.4754
Epoch 15/30
50000/50000 [==============================] - 13s 263us/step - loss: 1.4548 - acc: 0.4809 - val_loss: 1.5179 - val_acc: 0.4472
Epoch 16/30
50000/50000 [==============================] - 13s 263us/step - loss: 1.4428 - acc: 0.4853 - val_loss: 1.4551 - val_acc: 0.4843
Epoch 17/30
50000/50000 [==============================] - 13s 263us/step - loss: 1.4323 - acc: 0.4927 - val_loss: 1.5099 - val_acc: 0.4721
Epoch 18/30
50000/50

Epoch 29/30
50000/50000 [==============================] - 5s 105us/step - loss: 1.2094 - acc: 0.5715 - val_loss: 1.4251 - val_acc: 0.5063
Epoch 30/30
50000/50000 [==============================] - 5s 106us/step - loss: 1.2008 - acc: 0.5717 - val_loss: 1.5217 - val_acc: 0.4842
Test loss: 1.52170596466
Test accuracy: 0.4842
Parameters are epochs:  30   batch_size:  128   dropout_rate:  1    neurons: 256

<------------------------------------------------------------------------------------->

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_348 (Dense)            (None, 512)               1573376   
_________________________________________________________________
dropout_120 (Dropout)        (None, 512)               0         
_________________________________________________________________
dense_349 (Dense)            (None, 512)               262656    
__________________________________________

Epoch 10/30
50000/50000 [==============================] - 5s 92us/step - loss: 1.5847 - acc: 0.4341 - val_loss: 1.6072 - val_acc: 0.4294
Epoch 11/30
50000/50000 [==============================] - 5s 93us/step - loss: 1.5719 - acc: 0.4393 - val_loss: 1.6559 - val_acc: 0.4119
Epoch 12/30
50000/50000 [==============================] - 5s 92us/step - loss: 1.5566 - acc: 0.4434 - val_loss: 1.5340 - val_acc: 0.4531
Epoch 13/30
50000/50000 [==============================] - 5s 106us/step - loss: 1.5394 - acc: 0.4520 - val_loss: 1.5721 - val_acc: 0.4386
Epoch 14/30
50000/50000 [==============================] - 5s 98us/step - loss: 1.5259 - acc: 0.4584 - val_loss: 1.5886 - val_acc: 0.4408
Epoch 15/30
50000/50000 [==============================] - 5s 98us/step - loss: 1.5138 - acc: 0.4590 - val_loss: 1.5667 - val_acc: 0.4492
Epoch 16/30
50000/50000 [==============================] - 5s 98us/step - loss: 1.5053 - acc: 0.4619 - val_loss: 1.5084 - val_acc: 0.4652
Epoch 17/30
50000/50000 [========

Epoch 30/30
50000/50000 [==============================] - 9s 182us/step - loss: 1.2934 - acc: 0.5395 - val_loss: 1.4753 - val_acc: 0.4796
Test loss: 1.47527581081
Test accuracy: 0.4796
Parameters are epochs:  30   batch_size:  256   dropout_rate:  0.1    neurons: 512

<------------------------------------------------------------------------------------->

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_357 (Dense)            (None, 256)               786688    
_________________________________________________________________
dropout_126 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_358 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_127 (Dropout)        (None, 256)               0         
_______________________________________________

...

Epoch 40/40
50000/50000 [==============================] - 5s 92us/step - loss: 1.3436 - acc: 0.5204 - val_loss: 1.5587 - val_acc: 0.4474
Test loss: 1.55869760437
Test accuracy: 0.4474
Parameters are epochs:  40   batch_size:  256   dropout_rate:  0.1    neurons: 256

<------------------------------------------------------------------------------------->

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_390 (Dense)            (None, 512)               1573376   
_________________________________________________________________
dropout_148 (Dropout)        (None, 512)               0         
_________________________________________________________________
dense_391 (Dense)            (None, 512)               262656    
_________________________________________________________________
dropout_149 (Dropout)        (None, 512)               0         
________________________________________________

50000/50000 [==============================] - 9s 173us/step - loss: 2.3404 - acc: 0.2247 - val_loss: 2.1209 - val_acc: 0.2267
Epoch 2/40
50000/50000 [==============================] - 4s 85us/step - loss: 1.8708 - acc: 0.3251 - val_loss: 2.0594 - val_acc: 0.2778
Epoch 3/40
50000/50000 [==============================] - 4s 85us/step - loss: 1.7702 - acc: 0.3657 - val_loss: 1.6971 - val_acc: 0.3888
Epoch 4/40
50000/50000 [==============================] - 4s 85us/step - loss: 1.7026 - acc: 0.3916 - val_loss: 1.9334 - val_acc: 0.3132
Epoch 5/40
50000/50000 [==============================] - 4s 85us/step - loss: 1.6550 - acc: 0.4089 - val_loss: 1.6525 - val_acc: 0.4029
Epoch 6/40
50000/50000 [==============================] - 4s 85us/step - loss: 1.6144 - acc: 0.4220 - val_loss: 1.7442 - val_acc: 0.3727
Epoch 7/40
50000/50000 [==============================] - 4s 85us/step - loss: 1.5722 - acc: 0.4380 - val_loss: 1.7086 - val_acc: 0.3788
Epoch 8/40
50000/50000 [===========================

50000/50000 [==============================] - 4s 72us/step - loss: 1.4922 - acc: 0.4678 - val_loss: 1.4630 - val_acc: 0.4738
Epoch 22/40
50000/50000 [==============================] - 4s 72us/step - loss: 1.4813 - acc: 0.4724 - val_loss: 1.4488 - val_acc: 0.4847
Epoch 23/40
50000/50000 [==============================] - 4s 72us/step - loss: 1.4751 - acc: 0.4754 - val_loss: 1.4348 - val_acc: 0.4985
Epoch 24/40
50000/50000 [==============================] - 4s 72us/step - loss: 1.4651 - acc: 0.4770 - val_loss: 1.5946 - val_acc: 0.4448
Epoch 25/40
50000/50000 [==============================] - 4s 72us/step - loss: 1.4592 - acc: 0.4776 - val_loss: 1.4269 - val_acc: 0.4985
Epoch 26/40
50000/50000 [==============================] - 4s 72us/step - loss: 1.4471 - acc: 0.4848 - val_loss: 1.4216 - val_acc: 0.4999
Epoch 27/40
50000/50000 [==============================] - 4s 72us/step - loss: 1.4428 - acc: 0.4845 - val_loss: 1.4705 - val_acc: 0.4734
Epoch 28/40
50000/50000 [=====================

Epoch 31/40
50000/50000 [==============================] - 7s 147us/step - loss: 1.3203 - acc: 0.5307 - val_loss: 1.4034 - val_acc: 0.4999
Epoch 32/40
50000/50000 [==============================] - 7s 147us/step - loss: 1.3110 - acc: 0.5298 - val_loss: 1.4373 - val_acc: 0.4888
Epoch 33/40
50000/50000 [==============================] - 7s 147us/step - loss: 1.3045 - acc: 0.5344 - val_loss: 1.4193 - val_acc: 0.4945
Epoch 34/40
50000/50000 [==============================] - 7s 148us/step - loss: 1.2891 - acc: 0.5394 - val_loss: 1.3830 - val_acc: 0.5124
Epoch 35/40
50000/50000 [==============================] - 7s 150us/step - loss: 1.2814 - acc: 0.5419 - val_loss: 1.4315 - val_acc: 0.4988
Epoch 36/40
50000/50000 [==============================] - 8s 151us/step - loss: 1.2753 - acc: 0.5441 - val_loss: 1.4218 - val_acc: 0.4953
Epoch 37/40
50000/50000 [==============================] - 7s 149us/step - loss: 1.2639 - acc: 0.5457 - val_loss: 1.4154 - val_acc: 0.4941
Epoch 38/40
50000/50000 [==

dense_409 (Dense)            (None, 512)               262656    
_________________________________________________________________
dropout_161 (Dropout)        (None, 512)               0         
_________________________________________________________________
dense_410 (Dense)            (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 12s 242us/step - loss: 4.0570 - acc: 0.1777 - val_loss: 2.0767 - val_acc: 0.2093
Epoch 2/40
50000/50000 [==============================] - 7s 148us/step - loss: 1.9775 - acc: 0.2845 - val_loss: 1.8292 - val_acc: 0.3552
Epoch 3/40
50000/50000 [==============================] - 7s 147us/step - loss: 1.8735 - acc: 0.3286 - val_loss: 1.8302 - val_acc: 0.3255
Epoch 4/40
50000/50000 [==============================] - 7s 14

#### We can see when we ran the single layer model we got the best accuracy as 50.19% which is a 2% change from the previous two layer model
#### Most of the time the accuracy was in between 43% to 48%
#### As we increased the number of neurons the accuracy increased
#### increasing the number of epochs had a positive effect on the accuracy but not a major effect
#### Batch Size also had really minimal change on the test accuracy
#### Dropout rate when increased from 0.1 to 0.5 decreased the accuracy but as we went closet to the dropout raate 1 it started giving us the best results
#### One thing to note is that when we add a new layer it increases the time taken to build the model to almost double 
Since the 2% accuracy increase is not a lot it wont be advised to add a new layer considering the overall performance vs results tradeoff


#### So For our experiment I will suggest a two layer model with first dense layer using activation function of relu, dropout rate 1, batch size 512, epochs 40, neurons 512

Although the three layer model gave us the best results in terms of accuracy the overall tradeoff between time and increase in accuracy was not really good